In [1]:
import os 
import re

import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelBinarizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score, matthews_corrcoef, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
path_meta = './Original_Dataset/Meta'

In [3]:
dgt_df = pd.read_excel(os.path.join(path_meta, 'coffee_bean_Digital.xlsx'), engine='openpyxl')
dgt_df['Source'] = 'digital_camera'

and_df = pd.read_excel(os.path.join(path_meta, 'coffee_bean_Android.xlsx'), engine='openpyxl')
and_df['Source'] = 'android'

ios_df = pd.read_excel(os.path.join(path_meta, 'coffee_bean_iPhone.xlsx'), engine='openpyxl')
ios_df['Source'] = 'ios'

data_df = pd.concat([dgt_df, and_df, ios_df], axis=0)
data_df = data_df.iloc[:, :14].dropna()

In [4]:
data_df

,Name,Red,Green,Blue,Gray,Contrast,Energy,Homogeneity,Asm,Correlation,Dissimilarity,Group,pH,Source
0,BAL1,19.17,13.49,8.50,14.62,67.28,0.73,0.77,0.53,0.96,2.74,1.0,4.86,digital_camera
1,BAL2,17.23,12.19,7.54,13.16,61.78,0.73,0.77,0.53,0.95,2.52,1.0,4.86,digital_camera
2,BAL3,17.39,12.53,8.08,13.47,72.57,0.73,0.77,0.53,0.95,2.66,1.0,4.86,digital_camera
3,BAL4,16.62,11.73,7.32,12.69,62.59,0.75,0.78,0.56,0.95,2.46,1.0,4.86,digital_camera
4,BAL5,14.97,10.67,6.66,11.50,61.59,0.74,0.79,0.55,0.94,2.30,1.0,4.86,digital_camera
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,CD6,19.60,16.13,14.09,16.93,187.53,0.66,0.68,0.43,0.88,5.94,4.0,5.43,ios
116,CD7,10.34,8.11,6.52,8.59,95.70,0.69,0.72,0.48,0.81,4.10,4.0,5.43,ios
117,CD8,11.42,8.95,7.02,9.47,93.11,0.67,0.69,0.44,0.83,4.32,4.0,5.43,ios
118,CD9,10.79,8.41,6.69,8.93,87.49,0.66,0.70,0.44,0.82,4.10,4.0,5.43,ios


In [5]:
def real2class(row):
    if row['pH'] <= 5.0:
        return 0
    elif row['pH'] <= 5.2:
        return 1
    elif row['pH'] <= 5.5:
        return 2
    else:
        return 3

class_names = ['<=5.0', '(5.0,5.2]', '(5.2,5.5]', '>5.5']
num_classes = len(class_names)

data_df['class'] = data_df.apply(real2class, axis=1)
display(data_df)
display(data_df['class'].unique())

,Name,Red,Green,Blue,Gray,Contrast,Energy,Homogeneity,Asm,Correlation,Dissimilarity,Group,pH,Source,class
0,BAL1,19.17,13.49,8.50,14.62,67.28,0.73,0.77,0.53,0.96,2.74,1.0,4.86,digital_camera,0
1,BAL2,17.23,12.19,7.54,13.16,61.78,0.73,0.77,0.53,0.95,2.52,1.0,4.86,digital_camera,0
2,BAL3,17.39,12.53,8.08,13.47,72.57,0.73,0.77,0.53,0.95,2.66,1.0,4.86,digital_camera,0
3,BAL4,16.62,11.73,7.32,12.69,62.59,0.75,0.78,0.56,0.95,2.46,1.0,4.86,digital_camera,0
4,BAL5,14.97,10.67,6.66,11.50,61.59,0.74,0.79,0.55,0.94,2.30,1.0,4.86,digital_camera,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,CD6,19.60,16.13,14.09,16.93,187.53,0.66,0.68,0.43,0.88,5.94,4.0,5.43,ios,2
116,CD7,10.34,8.11,6.52,8.59,95.70,0.69,0.72,0.48,0.81,4.10,4.0,5.43,ios,2
117,CD8,11.42,8.95,7.02,9.47,93.11,0.67,0.69,0.44,0.83,4.32,4.0,5.43,ios,2
118,CD9,10.79,8.41,6.69,8.93,87.49,0.66,0.70,0.44,0.82,4.10,4.0,5.43,ios,2


array([0, 1, 2, 3])

In [24]:
X = data_df.drop(['Name', 'Source', 'pH', 'class', 'Group'], axis=1).to_numpy()
y = data_df[['Source', 'class']].to_numpy()

## Cross validation

In [225]:
skf = StratifiedKFold(n_splits=10)

In [247]:
model = RandomForestClassifier()
# model = GaussianNB()
# model = SVC(gamma='auto')
# model = MLPClassifier()

y_test_all = []
y_hat_all = []
fold = {'accuracy': [], 'precision': [], 'recall': [], 'f1': [], 'mcc': []}

for train_index, test_index in skf.split(X, y[:,1].astype(int)):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index][:,1].astype(int), y[test_index][:,1].astype(int)
    
    model.fit(X_train, y_train)
    y_hat = model.predict(X_test)
    
    print('Test Set')
    print(confusion_matrix(y_true=y_test, y_pred=y_hat))
    print(f'Accuracy: {accuracy_score(y_true=y_test, y_pred=y_hat):.2f}')
    print(f'Macro F1-score: {f1_score(y_true=y_test, y_pred=y_hat, average="macro"):.2f}')
    
    fold['accuracy'].append(accuracy_score(y_true=y_test, y_pred=y_hat))
    fold['precision'].append(precision_score(y_true=y_test, y_pred=y_hat, average='macro'))
    fold['recall'].append(recall_score(y_true=y_test, y_pred=y_hat, average='macro'))
    fold['f1'].append(f1_score(y_true=y_test, y_pred=y_hat, average='macro'))
    fold['mcc'].append(matthews_corrcoef(y_true=y_test, y_pred=y_hat))
    
    y_test_all.append(y_test)
    y_hat_all.append(y_hat)
    
y_test_all = np.hstack(y_test_all)
y_hat_all = np.hstack(y_hat_all)

TRAIN: [  6   7   8   9  10  11  12  13  14  15  16  17  18  19  28  29  42  43
  44  45  46  47  48  49  56  57  58  59  60  61  62  63  64  65  66  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85
  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103
 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121
 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139
 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157
 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175
 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193
 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211
 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229
 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247
 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265
 266 267 268 269 270 271 272 273 274 275 276

Test Set
[[2 0 3 1]
 [0 8 0 0]
 [0 6 2 4]
 [0 0 1 5]]
Accuracy: 0.53
Macro F1-score: 0.52
TRAIN: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95 100 101 102 103 104 105 106 107 108 109 110 111
 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 130 131
 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149
 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167
 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185
 190 191 192 193 194 195 196 197 198 199 202 203 204 205 206 207 208 209
 210 211 212 213 214 215 216 217 218 219 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 24

In [241]:
fold_result = pd.DataFrame(fold)
display(fold_result)

,accuracy,precision,recall,f1,mcc
0,0.34375,0.234659,0.395833,0.271039,0.158760
1,0.62500,0.497619,0.625000,0.541046,0.514802
2,0.53125,0.533333,0.500000,0.505037,0.345348
3,0.43750,0.350000,0.333333,0.267857,0.254353
4,0.31250,0.142857,0.218750,0.154167,-0.117579
5,0.37500,0.093750,0.250000,0.136364,0.000000
6,0.46875,0.294643,0.343750,0.275000,0.274352
7,0.43750,0.350000,0.312500,0.242857,0.240966
8,0.25000,0.126984,0.208333,0.153846,-0.091451
9,0.43750,0.231092,0.375000,0.285266,0.199156


In [242]:
fold_result.to_csv('./fold_result/combined_mlp_fold_result.csv', index=False)

In [248]:
print('Test Set')
print(confusion_matrix(y_true=y_test_all, y_pred=y_hat_all))
print(f'Accuracy: {accuracy_score(y_true=y_test_all, y_pred=y_hat_all):.2f}')
print(f'Precision: {precision_score(y_true=y_test_all, y_pred=y_hat_all, average="macro"):.2f}')
print(f'Recall: {recall_score(y_true=y_test_all, y_pred=y_hat_all, average="macro"):.2f}')
print(f'Macro F1-score: {f1_score(y_true=y_test_all, y_pred=y_hat_all, average="macro"):.2f}')
print(f'MCC: {matthews_corrcoef(y_true=y_test_all, y_pred=y_hat_all):.2f}')

Test Set
[[36 13 10  1]
 [15 47 17  1]
 [13 19 65 23]
 [ 1  4 26 29]]
Accuracy: 0.55
Precision: 0.55
Recall: 0.55
Macro F1-score: 0.55
MCC: 0.39


## Different Source

In [209]:
source = 'digital_camera'
X_train, X_test, y_train, y_test = train_test_split(
    X[y[:,0]==source], y[y[:,0]==source][:,1],
    random_state=42,
    stratify=y[y[:,0]==source][:,1],
    test_size=0.25)

X_train = np.concatenate((X_train, X[y[:,0]!=source]), axis=0)
y_train = np.concatenate((y_train, y[y[:,0]!=source][:,1]), axis=0)

y_test = y_test.astype(int)
y_train = y_train.astype(int)

In [216]:
# model = RandomForestClassifier()
# model = GaussianNB()
# model = SVC()
model = MLPClassifier()

model.fit(X_train, y_train)
y_hat = model.predict(X_test)

/home/pisol/anaconda3/envs/coffeebean/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [245]:
print('Test Set')
print(confusion_matrix(y_true=y_test, y_pred=y_hat))
print(f'Accuracy: {accuracy_score(y_true=y_test, y_pred=y_hat):.2f}')
print(f'Precision: {precision_score(y_true=y_test, y_pred=y_hat, average="macro"):.2f}')
print(f'Recall: {recall_score(y_true=y_test, y_pred=y_hat, average="macro"):.2f}')
print(f'Macro F1-score: {f1_score(y_true=y_test, y_pred=y_hat, average="macro"):.2f}')
print(f'MCC: {matthews_corrcoef(y_true=y_test, y_pred=y_hat):.2f}')

Test Set
[[1 1 4 0]
 [2 6 0 0]
 [0 6 4 2]
 [0 1 4 1]]
Accuracy: 0.38
Precision: 0.36
Recall: 0.35
Macro F1-score: 0.33
MCC: 0.13
